In [1]:
from sentence_transformers import SentenceTransformer
import os
import numpy as np
from pathlib import Path
import time

In [2]:
print(np.__version__) # sentence_transformer complaints if numpy > 1.19.5 is used

1.24.2


In [ ]:
# sentences = ["Det här är en exempelmening", "Varje exempel blir konverterad"]
# #model = SentenceTransformer('KBLab/sentence-bert-swedish-cased')
# model = SentenceTransformer("/home/max/Models/sentenceBERT/sts_fbmodel")
# embeddings = model.encode(sentences)
# print(embeddings.shape)
# print(np.array2string(embeddings))
# embeddings

In [3]:
os.listdir(Path("/home/max/Models/sentenceBERT"))

['sts_fbmodel_big_40epochs', 'fb_nli', 'sts_fbmodel']

In [ ]:
#"/home/max/Models/sentenceBERT/fb_nli".split("/")

In [4]:
def naming(full_path):
    return full_path.split("/")[-1]

In [5]:
def text2bert(corpus, models, out_dir, stop=None):
    
    t0 = time.time()
    
    corpus = Path(corpus)
    out_dir = Path(out_dir)
    
    for model in models:
        print()
        print(naming(model))
        sBERT = SentenceTransformer(model)
        for i, file in enumerate(sorted(os.listdir(corpus))):
            if stop != None:
                if i == stop:
                    return
            print(file.strip(".txt"), int((time.time()-t0)/60), "m.", end="\r")
            with open(corpus/file, "r") as f_in, open(out_dir/f"{naming(model)}/{file}", "w") as f_out:
                for line in f_in:
                    term, n, sentence = tuple([col for col in line.strip("\n").split("\t")])
                    vector = sBERT.encode(sentence)
                    as_str = " ".join([str(value) for value in vector.tolist()])
                    if int(n) < 2:
                        f_out.write(f"{term}\t{as_str}\n")
                    else:
                        for t in term.split("; "):
                            f_out.write(f"{t}\t{as_str}\n")
    
    print()
    print("Done!", int((time.time()-t0)/60), "m.")

In [6]:
where_my_models_are = "/home/max/Models/sentenceBERT"
models = set([f"{where_my_models_are}/{model}" for model in os.listdir(where_my_models_are)])

In [7]:
models

{'/home/max/Models/sentenceBERT/fb_nli',
 '/home/max/Models/sentenceBERT/sts_fbmodel',
 '/home/max/Models/sentenceBERT/sts_fbmodel_big_40epochs'}

In [8]:
models.add("KBLab/sentence-bert-swedish-cased")

In [9]:
models

{'/home/max/Models/sentenceBERT/fb_nli',
 '/home/max/Models/sentenceBERT/sts_fbmodel',
 '/home/max/Models/sentenceBERT/sts_fbmodel_big_40epochs',
 'KBLab/sentence-bert-swedish-cased'}

In [ ]:
#numpy.get_printoptions()['linewidth']

In [10]:
text2bert(
    corpus=Path("/home/max/Corpora/familjeliv-smh-time/yearly/contexts/files/"), 
    models=['KBLab/sentence-bert-swedish-cased'], 
    out_dir=Path("/home/max/Corpora/familjeliv-smh-time/yearly/contexts/vectors/")
)


sentence-bert-swedish-cased
files 7 m.
Done! 7 m.


In [ ]:
corpus = Path("/home/max/Corpora/flashback-pol-time/yearly/contexts/files")
out_dir = Path("/home/max/Corpora/flashback-pol-time/yearly/contexts/vectors")
text2bert(corpus, models, out_dir)
#text2bert(corpus, list(models)[:1], out_dir, stop = 2)

In [ ]:
# 16 kb --> 1.7 mb
#1700000 / 16000